In [1]:

from functools import reduce


class Grid:
    width = 101
    height = 103
    mid_height = (height // 2)
    mid_width = (width // 2)


class Robot:
    def __init__(self, position: tuple[int, int], heading: tuple[int, int]):
        self.position = position
        self.heading = heading

    def __repr__(self):
        return f'Robot(p={self.position}, v={self.heading})'


def load_input() -> list[Robot]:
    with open("../../data/day14-input.txt") as f:
        robots: list[Robot] = []
        for line in f.readlines():
            parts = line.split(' ')
            position = tuple([int(_) for _ in parts[0].replace('p=', '').split(',')])
            directions = tuple([int(_) for _ in parts[1].replace('v=', '').split(',')])
            robot = Robot(position, directions)
            robots.append(robot)
    return robots

In [2]:
def run(iterations: int):
    robots = load_input()
    for robot in robots:
        for i in range(0, iterations):
            robot.position = (robot.position[0] + robot.heading[0], robot.position[1] + robot.heading[1])
            x, y = robot.position
            robot.position = (x % Grid.width, y % Grid.height)

        #x, y = robot.position
        #robot.position = (x % Grid.width, y % Grid.height)

    quadrants = [0, 0, 0, 0]

    for robot in robots:
        if robot.position[0] != Grid.mid_width and robot.position[1] != Grid.mid_height:
            if robot.position[0] < Grid.mid_width and robot.position[1] < Grid.mid_height:
                quadrants[0] += 1
            elif robot.position[0] < Grid.mid_width and robot.position[1] > Grid.mid_height:
                quadrants[1] += 1
            elif robot.position[0] > Grid.mid_width and robot.position[1] < Grid.mid_height:
                quadrants[2] += 1
            else:
                quadrants[3] += 1
    return quadrants


quads = run(100)
print(reduce(lambda x, y: x * y, quads))


225943500


In [3]:
# too high: 229288752